In [1]:
import pandas as pd
import numpy

In [2]:
from sqlalchemy import create_engine

In [3]:
import datetime as dt
import time as t

In [4]:
#Create connection to sqlite
engine = create_engine('sqlite:///../db/candidates_tweets.sqlite', echo = False)

In [69]:
conn = engine.connect()
# Create dataframe from database
candidate_df = pd.read_sql("SELECT * FROM candidates_tweets", conn)

In [70]:
candidate_df.head()

,comments,date_string,datetime,engagement_score,favorites,name,party,retweets,stream_id,text,tweet_url,twitter_username,unix_time
0,255,3:35 PM - 6 May 2019,2019-05-06 15:35:00.000000,1485.836235,13685,Joe Biden,Democrat,940,1125529422205816834,Congratulations to the Duke and Duchess of Sus...,https://twitter.com/JoeBiden/status/1125529422...,JoeBiden,1557182100
1,292,2:38 PM - 6 May 2019,2019-05-06 14:38:00.000000,685.198576,2727,Joe Biden,Democrat,404,1125515077564502016,"On Saturday, Jill and I visited Columbia, Sout...",https://twitter.com/JoeBiden/status/1125515077...,JoeBiden,1557178680
2,151,1:28 PM - 6 May 2019,2019-05-06 13:28:00.000000,321.705558,1045,Joe Biden,Democrat,211,1125497461785616386,"Nevada, we’re taking our campaign to Henderson...",https://twitter.com/JoeBiden/status/1125497461...,JoeBiden,1557174480
3,511,5:31 PM - 5 May 2019,2019-05-05 17:31:00.000000,2371.390067,14870,Joe Biden,Democrat,1755,1125196320816467969,"On this first day of Ramadan, Jill and I wish ...",https://twitter.com/JoeBiden/status/1125196320...,JoeBiden,1557102682
4,521,4:17 PM - 5 May 2019,2019-05-05 16:17:00.000000,1040.328735,3626,Joe Biden,Democrat,596,1125177603780825089,"Our country may seem more divided than ever, b...",https://twitter.com/JoeBiden/status/1125177603...,JoeBiden,1557098220


In [71]:
extended_df = candidate_df

In [72]:
# extended_df.iterrows()

In [73]:
time_list = []
weekday_list = []

for i, r in extended_df.iterrows():
    date_string = r["date_string"]
    converted_object = dt.datetime.strptime(date_string, "%I:%M %p - %d %b %Y")
    
    time_object = dt.datetime.strftime(converted_object, "%H")
    time_list.append(time_object)
    
    weekday_object = dt.datetime.strftime(converted_object, "%a")
    weekday_list.append(weekday_object)
    
time_series = pd.Series(time_list)
weekday_series = pd.Series(weekday_list)

extended_df["tweet_hour"] = time_series
extended_df["tweet_weekday"] = weekday_series
    
# print(time_list[0:5])
# print(weekday_list[0:5])

extended_df.head()


['15', '14', '13', '17', '16']
['Mon', 'Mon', 'Mon', 'Sun', 'Sun']


,comments,date_string,datetime,engagement_score,favorites,name,party,retweets,stream_id,text,tweet_url,twitter_username,unix_time,tweet_hour,tweet_weekday
0,255,3:35 PM - 6 May 2019,2019-05-06 15:35:00.000000,1485.836235,13685,Joe Biden,Democrat,940,1125529422205816834,Congratulations to the Duke and Duchess of Sus...,https://twitter.com/JoeBiden/status/1125529422...,JoeBiden,1557182100,15,Mon
1,292,2:38 PM - 6 May 2019,2019-05-06 14:38:00.000000,685.198576,2727,Joe Biden,Democrat,404,1125515077564502016,"On Saturday, Jill and I visited Columbia, Sout...",https://twitter.com/JoeBiden/status/1125515077...,JoeBiden,1557178680,14,Mon
2,151,1:28 PM - 6 May 2019,2019-05-06 13:28:00.000000,321.705558,1045,Joe Biden,Democrat,211,1125497461785616386,"Nevada, we’re taking our campaign to Henderson...",https://twitter.com/JoeBiden/status/1125497461...,JoeBiden,1557174480,13,Mon
3,511,5:31 PM - 5 May 2019,2019-05-05 17:31:00.000000,2371.390067,14870,Joe Biden,Democrat,1755,1125196320816467969,"On this first day of Ramadan, Jill and I wish ...",https://twitter.com/JoeBiden/status/1125196320...,JoeBiden,1557102682,17,Sun
4,521,4:17 PM - 5 May 2019,2019-05-05 16:17:00.000000,1040.328735,3626,Joe Biden,Democrat,596,1125177603780825089,"Our country may seem more divided than ever, b...",https://twitter.com/JoeBiden/status/1125177603...,JoeBiden,1557098220,16,Sun


In [90]:
grouped_hour_df = extended_df.groupby(["name", "tweet_hour"])
hour_object = grouped_hour_df["engagement_score"].median()
hour_object.head()

name           tweet_hour
Amy Klobuchar  01            1537.776331
               02            1731.567620
               03             183.933420
               04             538.527128
               05             308.136381
Name: engagement_score, dtype: float64

In [92]:
hour_df = pd.DataFrame({
    "median_engagement": hour_object
})
hour_df.reset_index(inplace = True)
hour_df.head(10)

,name,tweet_hour,median_engagement
0,Amy Klobuchar,01,1537.776331
1,Amy Klobuchar,02,1731.567620
2,Amy Klobuchar,03,183.933420
3,Amy Klobuchar,04,538.527128
4,Amy Klobuchar,05,308.136381
5,Amy Klobuchar,06,283.555311
6,Amy Klobuchar,07,550.868029
7,Amy Klobuchar,08,381.383300
8,Amy Klobuchar,09,253.294123
9,Amy Klobuchar,10,331.816148


In [114]:
#Create connection to sqlite
hour_engine = create_engine('sqlite:///../db/candidates_tweets.sqlite', echo = False)

In [115]:
#Export data to sqlite
hour_df.to_sql('candidates_hour_categorized', con=hour_engine, if_exists='replace', index=False)

In [116]:
#Check if data was read into sqlite
hour_engine.execute("SELECT * from candidates_hour_categorized").fetchall()

[('Amy Klobuchar', '01', 1537.7763305298915),
 ('Amy Klobuchar', '02', 1731.5676202645013),
 ('Amy Klobuchar', '03', 183.93341951501142),
 ('Amy Klobuchar', '04', 538.527127808721),
 ('Amy Klobuchar', '05', 308.13638054261725),
 ('Amy Klobuchar', '06', 283.5553114739884),
 ('Amy Klobuchar', '07', 550.8680287756874),
 ('Amy Klobuchar', '08', 381.3832998447143),
 ('Amy Klobuchar', '09', 253.29412271796775),
 ('Amy Klobuchar', '10', 331.8161480158898),
 ('Amy Klobuchar', '11', 225.41868651995082),
 ('Amy Klobuchar', '12', 234.56781286733323),
 ('Amy Klobuchar', '13', 341.42772183482174),
 ('Amy Klobuchar', '14', 284.7339345138577),
 ('Amy Klobuchar', '15', 395.4270273664904),
 ('Amy Klobuchar', '16', 223.13543547566098),
 ('Amy Klobuchar', '17', 298.15846621868695),
 ('Amy Klobuchar', '18', 504.64558777339397),
 ('Amy Klobuchar', '19', 513.1174831339798),
 ('Amy Klobuchar', '20', 226.86175336667978),
 ('Amy Klobuchar', '21', 336.57985792965303),
 ('Andrew Yang', '00', 224.2163354298936),


In [93]:
grouped_day_df = extended_df.groupby(["name", "tweet_weekday"])
day_object = grouped_day_df["engagement_score"].median()
day_object.head()

name           tweet_weekday
Amy Klobuchar  Fri              350.084021
               Mon              277.349612
               Sat              190.291482
               Sun              322.539782
               Thu              354.836510
Name: engagement_score, dtype: float64

In [94]:
day_df = pd.DataFrame({
    "median_engagement": day_object
})
day_df.reset_index(inplace = True)
day_df.head(30)

,name,tweet_weekday,median_engagement
0,Amy Klobuchar,Fri,350.084021
1,Amy Klobuchar,Mon,277.349612
2,Amy Klobuchar,Sat,190.291482
3,Amy Klobuchar,Sun,322.539782
4,Amy Klobuchar,Thu,354.836510
5,Amy Klobuchar,Tue,309.553843
6,Amy Klobuchar,Wed,336.598582
7,Andrew Yang,Fri,228.577131
8,Andrew Yang,Mon,250.548938
9,Andrew Yang,Sat,239.619561


In [117]:
#Create connection to sqlite
day_engine = create_engine('sqlite:///../db/candidates_tweets.sqlite', echo = False)

In [120]:
#Export data to sqlite
day_df.to_sql('candidates_day_categorized', con=day_engine, if_exists = 'replace', index = False)

In [121]:
#Check if data was read into sqlite
day_engine.execute("SELECT * from candidates_day_categorized").fetchall()

[('Amy Klobuchar', 'Fri', 350.0840206448017),
 ('Amy Klobuchar', 'Mon', 277.3496118830702),
 ('Amy Klobuchar', 'Sat', 190.2914824274416),
 ('Amy Klobuchar', 'Sun', 322.5397823647975),
 ('Amy Klobuchar', 'Thu', 354.8365100523025),
 ('Amy Klobuchar', 'Tue', 309.5538428312699),
 ('Amy Klobuchar', 'Wed', 336.59858223035815),
 ('Andrew Yang', 'Fri', 228.5771312642627),
 ('Andrew Yang', 'Mon', 250.548937971874),
 ('Andrew Yang', 'Sat', 239.61956109386597),
 ('Andrew Yang', 'Sun', 239.6393767134935),
 ('Andrew Yang', 'Thu', 277.59890235725504),
 ('Andrew Yang', 'Tue', 256.0172616127474),
 ('Andrew Yang', 'Wed', 262.12153023779894),
 ('Bernie Sanders', 'Fri', 1411.6843996901607),
 ('Bernie Sanders', 'Mon', 1425.6261248028968),
 ('Bernie Sanders', 'Sat', 1395.0284284287409),
 ('Bernie Sanders', 'Sun', 1703.5680106198165),
 ('Bernie Sanders', 'Thu', 1280.130899292154),
 ('Bernie Sanders', 'Tue', 1724.5557005707415),
 ('Bernie Sanders', 'Wed', 1962.6151951266816),
 ("Beto O'Rourke", 'Fri', 465.23